# imports

In [1]:
import pandas as pd
import datetime as datetime

# 

# turning data into dataframes

In [2]:
alarms = pd.read_csv('alarms.csv', sep=';')
regions = pd.read_csv('regions.csv')
weather_by_hour = pd.read_csv('all_weather_by_hour_v2.csv')
isw_reports = pd.read_csv('result_data.csv')

In [3]:
alarms.head(5)

,id,region_id,region_title,region_city,all_region,start,end,clean_end,intersection_alarm_id
0,2,3,Вінниччина,Вінниця,0,2022-02-25 22:55:42,2022-02-25 23:41:53,2022-02-25 23:41:53,NaN
1,4,12,Львівщина,Львів,0,2022-02-26 06:26:17,2022-02-26 07:15:28,2022-02-26 07:15:28,NaN
2,5,14,Одещина,Одеса,0,2022-02-26 07:16:58,2022-02-26 07:47:03,2022-02-26 07:47:03,NaN
3,6,6,Житомирщина,Житомир,0,2022-02-26 08:05:54,2022-02-26 09:36:36,2022-02-26 09:36:36,NaN
4,7,3,Вінниччина,Вінниця,0,2022-02-26 08:39:39,2022-02-26 10:42:41,2022-02-26 10:42:41,NaN


In [4]:
regions.head(5)

,region,center_city_ua,center_city_en,region_alt,region_id
0,АР Крим,Сімферополь,Simferopol,Крим,1
1,Вінницька,Вінниця,Vinnytsia,Вінниччина,2
2,Волинська,Луцьк,Lutsk,Волинь,3
3,Дніпропетровська,Дніпро,Dnipro,Дніпропетровщина,4
4,Донецька,Донецьк,Donetsk,Донеччина,5


In [5]:
weather_by_hour.head(5)

,city_latitude,city_longitude,city_resolvedAddress,city_address,city_timezone,city_tzoffset,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,...,hour_visibility,hour_cloudcover,hour_solarradiation,hour_solarenergy,hour_uvindex,hour_severerisk,hour_conditions,hour_icon,hour_source,hour_stations
0,50.7469,25.3263,"Луцьк, Луцький район, Україна","Lutsk,Ukraine",Europe/Kiev,2.0,2022-02-24,1645653600,4.9,0.7,...,0.0,91.5,0.0,NaN,0.0,10.0,Overcast,snow,obs,remote
1,50.7469,25.3263,"Луцьк, Луцький район, Україна","Lutsk,Ukraine",Europe/Kiev,2.0,2022-02-24,1645653600,4.9,0.7,...,0.2,88.2,0.0,NaN,0.0,10.0,Partially cloudy,fog,obs,remote
2,50.7469,25.3263,"Луцьк, Луцький район, Україна","Lutsk,Ukraine",Europe/Kiev,2.0,2022-02-24,1645653600,4.9,0.7,...,10.0,100.0,NaN,NaN,NaN,10.0,Overcast,cloudy,obs,33177099999
3,50.7469,25.3263,"Луцьк, Луцький район, Україна","Lutsk,Ukraine",Europe/Kiev,2.0,2022-02-24,1645653600,4.9,0.7,...,0.1,92.0,0.0,NaN,0.0,10.0,Overcast,fog,obs,remote
4,50.7469,25.3263,"Луцьк, Луцький район, Україна","Lutsk,Ukraine",Europe/Kiev,2.0,2022-02-24,1645653600,4.9,0.7,...,0.0,93.8,0.0,NaN,0.0,10.0,Overcast,cloudy,obs,remote


In [6]:
isw_reports.head(5)

,date,tf-idf
0,2022-02-24,"{'abachev': 0, 'abandon': 0, 'abandoned': 0, '..."
1,2022-02-25,"{'abachev': 0, 'abandon': 0.028046167518242994..."
2,2022-02-26,"{'abachev': 0, 'abandon': 0.02043425393431761,..."
3,2022-02-27,"{'abachev': 0, 'abandon': 0, 'abandoned': 0, '..."
4,2022-02-28,"{'abachev': 0, 'abandon': 0, 'abandoned': 0, '..."


# 

# preparing alarms dataframe

### fixing region_id

In [7]:
alarms.loc[alarms['region_title'] == 'Вінниччина', 'region_id'] = 2
alarms.loc[alarms['region_title'] == 'Волинь', 'region_id'] = 3
alarms.loc[alarms['region_title'] == 'Івано-Франківщина', 'region_id'] = 9
alarms.loc[alarms['region_title'] == 'Буковина', 'region_id'] = 24
alarms.loc[alarms['region_title'] == 'Чернігівщина', 'region_id'] = 25

regions_to_update = ['Київщина', 'Кіровоградщина', 'Львівщина', 'Миколаївщина', 'Одещина',
                     'Полтавщина', 'Рівненщина', 'Сумщина', 'Тернопільщина', 'Харківщина',
                     'Херсонщина', 'Хмельниччина', 'Черкащина']

for region in regions_to_update:
    alarms.loc[alarms['region_title'] == region, 'region_id'] += 1

### changing data types & adding unix time

In [8]:
# to dt
alarms['full_start'] = pd.to_datetime(alarms['start'])
alarms['full_end'] = pd.to_datetime(alarms['end'])

# start-end hours
alarms['start_hour'] = alarms['full_start'].dt.floor('H')
alarms['end_hour'] = alarms['full_end'].dt.ceil('H')

# start day
alarms['day'] = alarms['full_start'].dt.date

# unix time
alarms['start_hour_dtEpoch'] = alarms['start_hour'].apply(lambda x: int(x.strftime('%s')))
alarms['end_hour_dtEpoch'] = alarms['end_hour'].apply(lambda x: int(x.strftime('%s')))

### droping useless columns

In [9]:
alarms = alarms.drop(['start'], axis=1)
alarms = alarms.drop(['end'], axis=1)
alarms = alarms.drop(['id'], axis=1)

### creating dataframe for alarms by hour

In [10]:
al_by_hour = []

for ind, alarm in alarms.iterrows():
    for i in pd.date_range(start=alarm['start_hour'], end=alarm['end_hour'], freq='1H'):
        al = alarm.copy()
        al['alarm_hour'] = i
        al_by_hour.append(al)

alarms_by_hour = pd.DataFrame(al_by_hour)

### changing data types & adding unix time

In [11]:
alarms_by_hour['alarm_hour'] = pd.to_datetime(alarms_by_hour['alarm_hour'])
# unix time
alarms_by_hour['alarm_hourEpoch'] = alarms_by_hour['alarm_hour'].apply(lambda x: int(x.strftime('%s')))

### done!

In [12]:
alarms_by_hour.head(5)

,region_id,region_title,region_city,all_region,clean_end,intersection_alarm_id,full_start,full_end,start_hour,end_hour,day,start_hour_dtEpoch,end_hour_dtEpoch,alarm_hour,alarm_hourEpoch
0,2,Вінниччина,Вінниця,0,2022-02-25 23:41:53,NaN,2022-02-25 22:55:42,2022-02-25 23:41:53,2022-02-25 22:00:00,2022-02-26 00:00:00,2022-02-25,1645819200,1645826400,2022-02-25 22:00:00,1645819200
0,2,Вінниччина,Вінниця,0,2022-02-25 23:41:53,NaN,2022-02-25 22:55:42,2022-02-25 23:41:53,2022-02-25 22:00:00,2022-02-26 00:00:00,2022-02-25,1645819200,1645826400,2022-02-25 23:00:00,1645822800
0,2,Вінниччина,Вінниця,0,2022-02-25 23:41:53,NaN,2022-02-25 22:55:42,2022-02-25 23:41:53,2022-02-25 22:00:00,2022-02-26 00:00:00,2022-02-25,1645819200,1645826400,2022-02-26 00:00:00,1645826400
1,13,Львівщина,Львів,0,2022-02-26 07:15:28,NaN,2022-02-26 06:26:17,2022-02-26 07:15:28,2022-02-26 06:00:00,2022-02-26 08:00:00,2022-02-26,1645848000,1645855200,2022-02-26 06:00:00,1645848000
1,13,Львівщина,Львів,0,2022-02-26 07:15:28,NaN,2022-02-26 06:26:17,2022-02-26 07:15:28,2022-02-26 06:00:00,2022-02-26 08:00:00,2022-02-26,1645848000,1645855200,2022-02-26 07:00:00,1645851600


# 

# preparing weather_by_hour dataframe

### droping useless columns

In [13]:
city_info = [
'city_latitude',
'city_longitude',
'city_address',
'city_timezone',
'city_tzoffset'
]

In [14]:
weather_by_hour2 = weather_by_hour.drop(city_info, axis=1)

### changing data types

In [15]:
weather_by_hour['hour_datetime'] = weather_by_hour['hour_datetime'].astype(str)
weather_by_hour['day_datetime'] = weather_by_hour['day_datetime'].astype(str)
weather_by_hour['datetime_combined'] = weather_by_hour['day_datetime'] + ' ' + weather_by_hour['hour_datetime']
weather_by_hour['datetime_combined'] = pd.to_datetime(weather_by_hour['datetime_combined'], format='%Y-%m-%d %H:%M:%S')

In [16]:
selected_columns = weather_by_hour[['datetime_combined']]
selected_columns.tail(100)

,datetime_combined
182612,2023-01-16 20:00:00
182613,2023-01-16 21:00:00
182614,2023-01-16 22:00:00
182615,2023-01-16 23:00:00
182616,2023-01-17 00:00:00
...,...
182707,2023-01-20 19:00:00
182708,2023-01-20 20:00:00
182709,2023-01-20 21:00:00
182710,2023-01-20 22:00:00


In [17]:
# weather_by_hour.columns

In [18]:
# to dt
weather_by_hour2['day_datetime'] = pd.to_datetime(weather_by_hour2['day_datetime'])
# weather_by_hour2['hour_datetime'] = pd.to_datetime(weather_by_hour2['hour_datetime'])

### creating new column for cities

In [19]:
weather_by_hour2['center_city'] = weather_by_hour2['city_resolvedAddress'].apply(lambda x: x.split(',')[0])
# fixing city name
weather_by_hour2['center_city'] = weather_by_hour2['center_city'].replace('Хмельницька область', 'Хмельницький')

# 

# merging weather data with regions

In [20]:
weather_regions = pd.merge(weather_by_hour2, regions, left_on='center_city',right_on='center_city_ua')

### done!

In [21]:
weather_regions.head(5)

,city_resolvedAddress,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_feelslikemax,day_feelslikemin,day_feelslike,day_dew,...,hour_conditions,hour_icon,hour_source,hour_stations,center_city,region,center_city_ua,center_city_en,region_alt,region_id
0,"Луцьк, Луцький район, Україна",2022-02-24,1645653600,4.9,0.7,2.6,4.0,-3.1,-0.2,0.0,...,Overcast,snow,obs,remote,Луцьк,Волинська,Луцьк,Lutsk,Волинь,3
1,"Луцьк, Луцький район, Україна",2022-02-24,1645653600,4.9,0.7,2.6,4.0,-3.1,-0.2,0.0,...,Partially cloudy,fog,obs,remote,Луцьк,Волинська,Луцьк,Lutsk,Волинь,3
2,"Луцьк, Луцький район, Україна",2022-02-24,1645653600,4.9,0.7,2.6,4.0,-3.1,-0.2,0.0,...,Overcast,cloudy,obs,33177099999,Луцьк,Волинська,Луцьк,Lutsk,Волинь,3
3,"Луцьк, Луцький район, Україна",2022-02-24,1645653600,4.9,0.7,2.6,4.0,-3.1,-0.2,0.0,...,Overcast,fog,obs,remote,Луцьк,Волинська,Луцьк,Lutsk,Волинь,3
4,"Луцьк, Луцький район, Україна",2022-02-24,1645653600,4.9,0.7,2.6,4.0,-3.1,-0.2,0.0,...,Overcast,cloudy,obs,remote,Луцьк,Волинська,Луцьк,Lutsk,Волинь,3


# 

# merging weather data with alarms data

In [22]:
alarms_by_hour.head(5)

,region_id,region_title,region_city,all_region,clean_end,intersection_alarm_id,full_start,full_end,start_hour,end_hour,day,start_hour_dtEpoch,end_hour_dtEpoch,alarm_hour,alarm_hourEpoch
0,2,Вінниччина,Вінниця,0,2022-02-25 23:41:53,NaN,2022-02-25 22:55:42,2022-02-25 23:41:53,2022-02-25 22:00:00,2022-02-26 00:00:00,2022-02-25,1645819200,1645826400,2022-02-25 22:00:00,1645819200
0,2,Вінниччина,Вінниця,0,2022-02-25 23:41:53,NaN,2022-02-25 22:55:42,2022-02-25 23:41:53,2022-02-25 22:00:00,2022-02-26 00:00:00,2022-02-25,1645819200,1645826400,2022-02-25 23:00:00,1645822800
0,2,Вінниччина,Вінниця,0,2022-02-25 23:41:53,NaN,2022-02-25 22:55:42,2022-02-25 23:41:53,2022-02-25 22:00:00,2022-02-26 00:00:00,2022-02-25,1645819200,1645826400,2022-02-26 00:00:00,1645826400
1,13,Львівщина,Львів,0,2022-02-26 07:15:28,NaN,2022-02-26 06:26:17,2022-02-26 07:15:28,2022-02-26 06:00:00,2022-02-26 08:00:00,2022-02-26,1645848000,1645855200,2022-02-26 06:00:00,1645848000
1,13,Львівщина,Львів,0,2022-02-26 07:15:28,NaN,2022-02-26 06:26:17,2022-02-26 07:15:28,2022-02-26 06:00:00,2022-02-26 08:00:00,2022-02-26,1645848000,1645855200,2022-02-26 07:00:00,1645851600


In [23]:
weather_regions.head(5)

,city_resolvedAddress,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_feelslikemax,day_feelslikemin,day_feelslike,day_dew,...,hour_conditions,hour_icon,hour_source,hour_stations,center_city,region,center_city_ua,center_city_en,region_alt,region_id
0,"Луцьк, Луцький район, Україна",2022-02-24,1645653600,4.9,0.7,2.6,4.0,-3.1,-0.2,0.0,...,Overcast,snow,obs,remote,Луцьк,Волинська,Луцьк,Lutsk,Волинь,3
1,"Луцьк, Луцький район, Україна",2022-02-24,1645653600,4.9,0.7,2.6,4.0,-3.1,-0.2,0.0,...,Partially cloudy,fog,obs,remote,Луцьк,Волинська,Луцьк,Lutsk,Волинь,3
2,"Луцьк, Луцький район, Україна",2022-02-24,1645653600,4.9,0.7,2.6,4.0,-3.1,-0.2,0.0,...,Overcast,cloudy,obs,33177099999,Луцьк,Волинська,Луцьк,Lutsk,Волинь,3
3,"Луцьк, Луцький район, Україна",2022-02-24,1645653600,4.9,0.7,2.6,4.0,-3.1,-0.2,0.0,...,Overcast,fog,obs,remote,Луцьк,Волинська,Луцьк,Lutsk,Волинь,3
4,"Луцьк, Луцький район, Україна",2022-02-24,1645653600,4.9,0.7,2.6,4.0,-3.1,-0.2,0.0,...,Overcast,cloudy,obs,remote,Луцьк,Волинська,Луцьк,Lutsk,Волинь,3


In [24]:
alarms_weather = weather_regions.merge(alarms_by_hour, how="left", 
                                       left_on=['region_alt','hour_datetimeEpoch'],
                                       right_on=['region_title','alarm_hourEpoch'])

### done!

In [25]:
alarms_weather[alarms_weather['alarm_hourEpoch']>0].head(5)

,city_resolvedAddress,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_feelslikemax,day_feelslikemin,day_feelslike,day_dew,...,intersection_alarm_id,full_start,full_end,start_hour,end_hour,day,start_hour_dtEpoch,end_hour_dtEpoch,alarm_hour,alarm_hourEpoch
64,"Луцьк, Луцький район, Україна",2022-02-26,1645826400,6.3,-1.5,1.9,4.5,-4.1,0.6,-2.7,...,NaN,2022-02-26 16:08:26,2022-02-26 16:39:26,2022-02-26 16:00:00,2022-02-26 17:00:00,2022-02-26,1.645884e+09,1.645888e+09,2022-02-26 16:00:00,1.645884e+09
65,"Луцьк, Луцький район, Україна",2022-02-26,1645826400,6.3,-1.5,1.9,4.5,-4.1,0.6,-2.7,...,NaN,2022-02-26 16:08:26,2022-02-26 16:39:26,2022-02-26 16:00:00,2022-02-26 17:00:00,2022-02-26,1.645884e+09,1.645888e+09,2022-02-26 17:00:00,1.645888e+09
89,"Луцьк, Луцький район, Україна",2022-02-27,1645912800,2.5,-1.9,0.0,-0.1,-6.1,-1.8,-3.2,...,NaN,2022-02-27 17:52:22,2022-02-27 18:11:37,2022-02-27 17:00:00,2022-02-27 19:00:00,2022-02-27,1.645974e+09,1.645981e+09,2022-02-27 17:00:00,1.645974e+09
90,"Луцьк, Луцький район, Україна",2022-02-27,1645912800,2.5,-1.9,0.0,-0.1,-6.1,-1.8,-3.2,...,NaN,2022-02-27 17:52:22,2022-02-27 18:11:37,2022-02-27 17:00:00,2022-02-27 19:00:00,2022-02-27,1.645974e+09,1.645981e+09,2022-02-27 18:00:00,1.645978e+09
91,"Луцьк, Луцький район, Україна",2022-02-27,1645912800,2.5,-1.9,0.0,-0.1,-6.1,-1.8,-3.2,...,NaN,2022-02-27 17:52:22,2022-02-27 18:11:37,2022-02-27 17:00:00,2022-02-27 19:00:00,2022-02-27,1.645974e+09,1.645981e+09,2022-02-27 19:00:00,1.645981e+09


#

# merging alarms_weather data with isw_reports

In [26]:
isw_reports.head(5)

,date,tf-idf
0,2022-02-24,"{'abachev': 0, 'abandon': 0, 'abandoned': 0, '..."
1,2022-02-25,"{'abachev': 0, 'abandon': 0.028046167518242994..."
2,2022-02-26,"{'abachev': 0, 'abandon': 0.02043425393431761,..."
3,2022-02-27,"{'abachev': 0, 'abandon': 0, 'abandoned': 0, '..."
4,2022-02-28,"{'abachev': 0, 'abandon': 0, 'abandoned': 0, '..."


In [27]:
# to dt
isw_reports['date'] = pd.to_datetime(isw_reports['date'])

In [28]:
main_dataset = alarms_weather.merge(isw_reports,
                      how="left",
                      left_on='day_datetime',
                      right_on='date')

### done!

In [29]:
main_dataset[main_dataset['alarm_hourEpoch']>0].head(5)
# main_dataset.head(5)

,city_resolvedAddress,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_feelslikemax,day_feelslikemin,day_feelslike,day_dew,...,full_end,start_hour,end_hour,day,start_hour_dtEpoch,end_hour_dtEpoch,alarm_hour,alarm_hourEpoch,date,tf-idf
64,"Луцьк, Луцький район, Україна",2022-02-26,1645826400,6.3,-1.5,1.9,4.5,-4.1,0.6,-2.7,...,2022-02-26 16:39:26,2022-02-26 16:00:00,2022-02-26 17:00:00,2022-02-26,1.645884e+09,1.645888e+09,2022-02-26 16:00:00,1.645884e+09,2022-02-26,"{'abachev': 0, 'abandon': 0.02043425393431761,..."
65,"Луцьк, Луцький район, Україна",2022-02-26,1645826400,6.3,-1.5,1.9,4.5,-4.1,0.6,-2.7,...,2022-02-26 16:39:26,2022-02-26 16:00:00,2022-02-26 17:00:00,2022-02-26,1.645884e+09,1.645888e+09,2022-02-26 17:00:00,1.645888e+09,2022-02-26,"{'abachev': 0, 'abandon': 0.02043425393431761,..."
89,"Луцьк, Луцький район, Україна",2022-02-27,1645912800,2.5,-1.9,0.0,-0.1,-6.1,-1.8,-3.2,...,2022-02-27 18:11:37,2022-02-27 17:00:00,2022-02-27 19:00:00,2022-02-27,1.645974e+09,1.645981e+09,2022-02-27 17:00:00,1.645974e+09,2022-02-27,"{'abachev': 0, 'abandon': 0, 'abandoned': 0, '..."
90,"Луцьк, Луцький район, Україна",2022-02-27,1645912800,2.5,-1.9,0.0,-0.1,-6.1,-1.8,-3.2,...,2022-02-27 18:11:37,2022-02-27 17:00:00,2022-02-27 19:00:00,2022-02-27,1.645974e+09,1.645981e+09,2022-02-27 18:00:00,1.645978e+09,2022-02-27,"{'abachev': 0, 'abandon': 0, 'abandoned': 0, '..."
91,"Луцьк, Луцький район, Україна",2022-02-27,1645912800,2.5,-1.9,0.0,-0.1,-6.1,-1.8,-3.2,...,2022-02-27 18:11:37,2022-02-27 17:00:00,2022-02-27 19:00:00,2022-02-27,1.645974e+09,1.645981e+09,2022-02-27 19:00:00,1.645981e+09,2022-02-27,"{'abachev': 0, 'abandon': 0, 'abandoned': 0, '..."


#

# cleaning main dataframe

In [30]:
# for model training
main_dataset['alarm'] = main_dataset['full_start'].apply(lambda x: 0 if pd.isnull(x) else 1)

In [31]:
main_dataset.shape

(195085, 86)

In [32]:
main_dataset.columns

Index(['city_resolvedAddress', 'day_datetime', 'day_datetimeEpoch',
       'day_tempmax', 'day_tempmin', 'day_temp', 'day_feelslikemax',
       'day_feelslikemin', 'day_feelslike', 'day_dew', 'day_humidity',
       'day_precip', 'day_precipprob', 'day_precipcover', 'day_snow',
       'day_snowdepth', 'day_windgust', 'day_windspeed', 'day_winddir',
       'day_pressure', 'day_cloudcover', 'day_visibility',
       'day_solarradiation', 'day_solarenergy', 'day_uvindex',
       'day_severerisk', 'day_sunrise', 'day_sunriseEpoch', 'day_sunset',
       'day_sunsetEpoch', 'day_moonphase', 'day_conditions', 'day_description',
       'day_icon', 'day_source', 'day_preciptype', 'day_stations',
       'hour_datetime', 'hour_datetimeEpoch', 'hour_temp', 'hour_feelslike',
       'hour_humidity', 'hour_dew', 'hour_precip', 'hour_precipprob',
       'hour_snow', 'hour_snowdepth', 'hour_preciptype', 'hour_windgust',
       'hour_windspeed', 'hour_winddir', 'hour_pressure', 'hour_visibility',
       'h

In [33]:
colums_to_drop = [
    'day_datetimeEpoch',
    'day_sunriseEpoch',
    'day_sunsetEpoch',
    'hour_datetimeEpoch',
    'start_hour_dtEpoch',
    'end_hour_dtEpoch',
    'alarm_hourEpoch',
    'city_resolvedAddress',
    'region',
    'region_alt',
    'region_title',
    'center_city_en',
    'center_city_ua',
    'center_city',
    'region_city',
    'all_region',
    'clean_end',
    'full_start',
    'full_end',
    'start_hour',
    'end_hour',
    'intersection_alarm_id',
    'day',
    'date',
    'alarm_hour',
    'region_id_y'
]

In [34]:
weather_info = [
    'hour_icon',
    'hour_source',
    'hour_stations',
    'hour_feelslike',
    'hour_preciptype',
    'hour_conditions',
    'hour_solarenergy',
    'day_severerisk',
    'day_feelslikemax',
    'day_feelslikemin',
    'day_feelslike',
    'day_description',
    'day_conditions',
    'day_icon',
    'day_precipprob',
    'day_preciptype',
    'day_stations',
    'day_source',
    'day_snowdepth',
    'day_sunset',
    'day_sunrise'
]

In [35]:
main_dataset = main_dataset.drop(colums_to_drop, axis=1)

In [36]:
main_dataset = main_dataset.drop(weather_info, axis=1)

In [37]:
main_dataset.columns

Index(['day_datetime', 'day_tempmax', 'day_tempmin', 'day_temp', 'day_dew',
       'day_humidity', 'day_precip', 'day_precipcover', 'day_snow',
       'day_windgust', 'day_windspeed', 'day_winddir', 'day_pressure',
       'day_cloudcover', 'day_visibility', 'day_solarradiation',
       'day_solarenergy', 'day_uvindex', 'day_moonphase', 'hour_datetime',
       'hour_temp', 'hour_humidity', 'hour_dew', 'hour_precip',
       'hour_precipprob', 'hour_snow', 'hour_snowdepth', 'hour_windgust',
       'hour_windspeed', 'hour_winddir', 'hour_pressure', 'hour_visibility',
       'hour_cloudcover', 'hour_solarradiation', 'hour_uvindex',
       'hour_severerisk', 'region_id_x', 'tf-idf', 'alarm'],
      dtype='object')

In [38]:
main_dataset['hour_datetime'] = main_dataset['hour_datetime'].astype(str)
main_dataset['day_datetime'] = main_dataset['day_datetime'].astype(str)
main_dataset['datetime_combined'] = main_dataset['day_datetime'] + ' ' + main_dataset['hour_datetime']
main_dataset['datetime_combined'] = pd.to_datetime(main_dataset['datetime_combined'], format='%Y-%m-%d %H:%M:%S')

In [39]:
to_drop = [
    'hour_datetime',
    'day_datetime'
]

In [44]:
main_dataset['datetime_combined'].dtype

dtype('<M8[ns]')

In [40]:
main_dataset = main_dataset.drop(to_drop, axis=1)

### done!

In [41]:
main_dataset

,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_snow,day_windgust,day_windspeed,...,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_uvindex,hour_severerisk,region_id_x,tf-idf,alarm,datetime_combined
0,4.9,0.7,2.6,0.0,83.7,0.118,4.17,0.1,32.4,15.5,...,1020.0,0.0,91.5,0.0,0.0,10.0,3,"{'abachev': 0, 'abandon': 0, 'abandoned': 0, '...",0,2022-02-24 00:00:00
1,4.9,0.7,2.6,0.0,83.7,0.118,4.17,0.1,32.4,15.5,...,1021.0,0.2,88.2,0.0,0.0,10.0,3,"{'abachev': 0, 'abandon': 0, 'abandoned': 0, '...",0,2022-02-24 01:00:00
2,4.9,0.7,2.6,0.0,83.7,0.118,4.17,0.1,32.4,15.5,...,1022.0,10.0,100.0,NaN,NaN,10.0,3,"{'abachev': 0, 'abandon': 0, 'abandoned': 0, '...",0,2022-02-24 02:00:00
3,4.9,0.7,2.6,0.0,83.7,0.118,4.17,0.1,32.4,15.5,...,1021.0,0.1,92.0,0.0,0.0,10.0,3,"{'abachev': 0, 'abandon': 0, 'abandoned': 0, '...",0,2022-02-24 03:00:00
4,4.9,0.7,2.6,0.0,83.7,0.118,4.17,0.1,32.4,15.5,...,1021.0,0.0,93.8,0.0,0.0,10.0,3,"{'abachev': 0, 'abandon': 0, 'abandoned': 0, '...",0,2022-02-24 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195080,11.2,5.7,7.9,6.2,89.3,0.600,12.50,0.0,34.9,16.6,...,1010.0,24.1,100.0,0.0,0.0,10.0,2,"{'abachev': 0, 'abandon': 0, 'abandoned': 0, '...",0,2023-01-20 19:00:00
195081,11.2,5.7,7.9,6.2,89.3,0.600,12.50,0.0,34.9,16.6,...,1010.8,10.0,30.0,0.0,0.0,10.0,2,"{'abachev': 0, 'abandon': 0, 'abandoned': 0, '...",0,2023-01-20 20:00:00
195082,11.2,5.7,7.9,6.2,89.3,0.600,12.50,0.0,34.9,16.6,...,1011.0,24.1,100.0,0.0,0.0,10.0,2,"{'abachev': 0, 'abandon': 0, 'abandoned': 0, '...",0,2023-01-20 21:00:00
195083,11.2,5.7,7.9,6.2,89.3,0.600,12.50,0.0,34.9,16.6,...,1011.0,24.1,100.0,0.0,0.0,10.0,2,"{'abachev': 0, 'abandon': 0, 'abandoned': 0, '...",0,2023-01-20 22:00:00


In [42]:
main_dataset.shape

(195085, 38)

#

### turning to csv

In [43]:
main_dataset.to_csv('main_dataset.csv')